## Damian Skowroński | Warszaty badawcze | Praca domowa 2

Wczytanie przetworzonych danych z pierwszej pracy domowej

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("PD1_data.csv", index_col = 0)

W początkowej ramce danych zmienna _readmitted_ miała trzy możliwe wartości "NO",">30" i "<30". Po przetworzeniu danych w pracy domowej 1 te wartości mają wartości odpowiednio 2 ,1 ,0. Teraz zamieniam je, żeby była to zmienna binarna o wartościach 0 lub 1. gdzie 0 oznacza, że pacjent był ponownie przyjęty, a 1 oznacza, że nie był.  

In [2]:
df["readmitted"] = df["readmitted"].apply(lambda x: 0 if x == 2 else 1)
df.readmitted.value_counts()

0    52338
1    45715
Name: readmitted, dtype: int64

Dodatkowo wykonam encoding zmiennych _diag_1_, _diag_2_ i _diag_3_, używając Label Encoding, ponieważ nie zrobiłem tego wcześniej.

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in ["diag_1","diag_2","diag_3"]:
    df[col] = label_encoder.fit_transform(df[col])

Podział zbioru na zbiory treningowe i testowe.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != "readmitted"], df["readmitted"], test_size = 0.25, random_state=123
)

Użyję modelu _Random Forest Classifier_. Na początku dla domyślnych parametrów.

In [5]:
from sklearn.ensemble import RandomForestClassifier
default_model = RandomForestClassifier()

Używam _cross validatoin_ dla z podziałem na 10 zbiorów. Jako metryki stosuję _AOC_ i _F1_. Piszę funkcję printującą wyniki.

In [26]:
from sklearn.model_selection import cross_validate

def get_scores(model):
    cv = cross_validate(default_model,X_train,y_train,cv = 5,scoring = ["roc_auc","f1"])
    print("AUC = ",max(cv["test_roc_auc"]),"\nF1 = ", max(cv["test_f1"]))

I liczę dla modelu domyślego

In [27]:
get_scores(default_model)

AUC =  0.6853062922708105 
F1 =  0.5772012578616352


### Ręczne dobieranie hiperparametrów

Z szybkiego researchu wygląda na to, że potencjalnymi parametrami, które mogą mieć duży wpływ na model są:

*   n_estimators
*   criterion
*   max_depth
*   min_samples_split
*   max_samples_leaf
*   max_features


Sprawdzam dla różnych wartości parametru _n_estimators_:

In [30]:
for val in [10,50,200,500]: #domyślnie jest 100
    rfc = RandomForestClassifier(n_estimators=val)
    print("Dla n_estimators = ", val, ":")
    get_scores(rfc)

Dla n_estimators =  10 :
AUC =  0.6836949353068168 
F1 =  0.5760436987904799
Dla n_estimators =  50 :
AUC =  0.6869140094917441 
F1 =  0.581761252446184
Dla n_estimators =  200 :
AUC =  0.6836775815318694 
F1 =  0.5788150243595788
Dla n_estimators =  500 :
AUC =  0.6854796628892262 
F1 =  0.5767054385413392


Wynik okazał się najlepszy dla _n_estimators_ = 50, jednak nie jest to jakas duża zmiana względem modelu domyślego. 

Sprawdzam dla parametru _criterion_, które domyślnie jest jako "gini", a może przyjąć też wartość "entropy". 

In [29]:
rfc = RandomForestClassifier(criterion="entropy")
get_scores(rfc)

AUC =  0.6853194677692859 
F1 =  0.5762310606060606


Metryki są bardzo podobne do domyślnego modelu, i czas wykonania jest taki sam więc myślę, że nie warto się zajmować tym parametrem.

Sprawdzam dla parametru _max_depth_.

In [32]:
for val in [5,10,15,30,50]: #default = None
    rfc = RandomForestClassifier(max_depth=val)
    print("\nDla max_depth = ",val, ":")
    get_scores(rfc)


Dla max_depth =  5 :
AUC =  0.6864054389646194 
F1 =  0.5773907586098691

Dla max_depth =  10 :
AUC =  0.686005986448828 
F1 =  0.5808331364674384

Dla max_depth =  15 :
AUC =  0.6860584284610823 
F1 =  0.576173845556849

Dla max_depth =  30 :
AUC =  0.6831955477028717 
F1 =  0.5734795298572217

Dla max_depth =  50 :
AUC =  0.6868356343101865 
F1 =  0.574254292269958


Wartości są znowu bardzo podobne. AUC najlepiej wyszło dla _max_depth_ = 50, ale dobrym wynikiem jest też dla _max_depth_ = 5

Sprawdzam dla _min_samples_split_.

In [33]:
for val in range(4,11,2): #default = 2
    rfc = RandomForestClassifier(min_samples_split=val)
    print("\nDla min_samples_split = ",val, ":")
    get_scores(rfc)


Dla min_samples_split =  4 :
AUC =  0.6864417342595737 
F1 =  0.575201644788866

Dla min_samples_split =  6 :
AUC =  0.6851776199255877 
F1 =  0.5761589403973509

Dla min_samples_split =  8 :
AUC =  0.6852861436931562 
F1 =  0.5756545123783912


Znowu zmiana dla AUC nie wydaje się duża. Najlepsze AUC jest dla _min_samples_split_ = 4, a reszta wyników jest gorsza niż dla domyślej wartości.

Sprawdzam dla _min_samples_leaf_.

In [34]:
for val in range(2,11,2): #default = 1
    rfc = RandomForestClassifier(min_samples_leaf=val)
    print("\nDla min_samples_split = ",val, ":")
    get_scores(rfc)     


Dla min_samples_split =  2 :
AUC =  0.6857956612962813 
F1 =  0.5761407366684992

Dla min_samples_split =  4 :
AUC =  0.6860142687213016 
F1 =  0.575521448248721

Dla min_samples_split =  6 :
AUC =  0.6857977875747527 
F1 =  0.5796418473138549

Dla min_samples_split =  8 :
AUC =  0.6849891053762256 
F1 =  0.5772351319417094

Dla min_samples_split =  10 :
AUC =  0.6875698781920989 
F1 =  0.5789348737833347


Po raz kolejny zmienna nie wydaje się mieć dużego wpływu na wyniki.

Podsumowując, podczas ręcznego sprawdzania, nie znalazłem parametrów, któe mocno by się wyróżniały pod względem otrzymywanych wyników.

### Szukanie hiperparametrów używając _random search_, _grid search_ i _optymalizacji bayesowskiej_

#### 1. Random Search

In [37]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

model = RandomForestClassifier()
randomized = RandomizedSearchCV(model,param_distributions={
    "n_estimators" : randint(50,500),
    "min_samples_split" : randint(3,10),
    "max_depth" : randint(10,20)
}, n_jobs=-1)

cv_randomized = cross_validate(randomized,X_train,y_train,cv = 5,scoring = ["roc_auc","f1"])

Najlepszy otrzymany wynik AUC wynosi:

In [57]:
max(cv_randomized["test_roc_auc"])

0.696258669111237

#### 2. Grid Search


In [43]:
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()
grid = GridSearchCV(model, param_grid={
    'n_estimators': [50, 500, 1000], 
    'min_samples_split': [4,10],
    'max_depth': [10,15]
    }, n_jobs=-1, cv = 5,scoring="roc_auc")

grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 15], 'min_samples_split': [4, 10],
                         'n_estimators': [50, 500, 1000]},
             scoring='roc_auc')

Najlepszy wynik AUC:

In [47]:
grid.best_score_

0.691885674128556

Dla parametrów:

In [48]:
grid.best_estimator_

RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=500)

#### 3. Optymalizacja Bayesowska

In [54]:
from skopt import BayesSearchCV
from skopt.space import Integer

model = RandomForestClassifier()
bayes = BayesSearchCV(model,search_spaces={
    "n_estimators" : Integer(50,500),
    "min_samples_split" : Integer(3,10),
    "max_depth" : Integer(10,20)
},n_jobs=-1,cv = 5,scoring="roc_auc")

bayes.fit(X_train,y_train)

C:\Users\skdam\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\skdam\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\skdam\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\skdam\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
              scoring='roc_auc',
              search_spaces={'max_depth': Integer(low=10, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=3, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=50, high=500, prior='uniform', transform='normalize')})

In [56]:
bayes.best_estimator_

RandomForestClassifier(max_depth=18, min_samples_split=10, n_estimators=500)

In [55]:
bayes.best_score_

0.6918972568649056

## Podsumowanie
Już w trakcie ręcznego dopasowywania parametrów, często otrzymywałem wynik AUC lepszy niż dla domyślnego modelu. Spośród algorytmów automatycznie dobierających parametry najlepszy wyszedł dla _random search_, a dla _grid search_ i _optymalizacji bayesa_ wyszło bardzo podobnie, z tym że ta druga działała 2 razy dłużej. Wyniki pomiędzy domyślnym modelem, a najlepszymi jakie udało mi się otrzymać, niewiele się różnią, bo o około 0.01.